In [ ]:
import numpy as np
import pickle
import os
import pyimfit
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import angular_separation
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from photutils.isophote import EllipseSample, Isophote
from photutils.isophote.sample import CentralEllipseSample
from photutils.isophote.fitter import CentralEllipseFitter
from photutils.isophote import Ellipse, EllipseGeometry, IsophoteList
from matplotlib.ticker import FormatStrFormatter
import glob
import matplotlib.gridspec as gridspec
from photutils.aperture import EllipticalAperture
from photutils.detection import find_peaks
import astropy.units as u
from astropy.cosmology import WMAP9 as cosmo
import seaborn as sns

medium_font_size = 14 
plt.rcParams['font.size'] = medium_font_size
plt.rcParams['axes.labelsize'] = medium_font_size
plt.rcParams['axes.titlesize'] = medium_font_size
plt.rcParams['xtick.labelsize'] = medium_font_size
plt.rcParams['ytick.labelsize'] = medium_font_size
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['font.family'] = 'monospace'
# fl = os.listdir("../cutouts/data")
# agns = sorted([f[:-5] for f in fl if f[0]=="J"])

In [ ]:
def find_highest_indices(arr):
    """returns a tuple of ys, xs - indices of pixels with highest counts"""
    flattened_arr = np.array(arr).flatten()
    max_indices = np.unravel_index(np.argsort(flattened_arr)[-2:], arr.shape)
    return max_indices

def makeModelDict(PA_ss, ell_ss, n_ss, I_ss, r_ss, Itot,
                 PA_lim, ell_lim, I_lim,  Iss_lim, rss_lim, Itot_lim,
                 sigma, sigma_lim, Isky, Isky_lim,
                 h1,h2,h_lim,alpha,alpha_lim):
    """Return Sersic, PSF, and Gaussian model parameter dictionary"""
    # Sersic
    """sersic = {'PA': [PA_ss, PA_lim[0],PA_lim[1]], 'ell_bulge': [ell_ss, ell_lim[0],ell_lim[1]], 'n': [n_ss, 'fixed'],
    'I_e': [I_ss, Iss_lim[0],Iss_lim[1]], 'r_e': [r_ss, rss_lim[0],rss_lim[1]]}"""
    sersic = {'PA': [PA_ss, PA_lim[0],PA_lim[1]], 'ell_bulge': [ell_ss, ell_lim[0],ell_lim[1]], 'n': [n_ss, 0.3, 6],
    'I_e': [I_ss, Iss_lim[0],Iss_lim[1]], 'r_e': [r_ss, rss_lim[0],rss_lim[1]]}
    sersic_dict = {'name': "Sersic", 'label': "bulge", 'parameters': sersic}
    # PSF
    psf = {'I_tot' : [Itot, Itot_lim[0], Itot_lim[1]]}
    psf_dict = {'name': "PointSource", 'label': "psf", 'parameters': psf}
    # Rot psf
    psfRot = {'I_tot' : [Itot, Itot_lim[0], Itot_lim[1]], 'PA':[PA_ss, PA_lim[0],PA_lim[1]] }
    psfRot_dict = {'name': "PointSourceRot", 'label': "psf", 'parameters': psfRot}
    # Gaussians
    gaussian = {'PA':[PA_ss, PA_lim[0],PA_lim[1]], 'ell':[ell_ss, ell_lim[0],ell_lim[1]], 
                'I_0':[I_ss, Iss_lim[0],Iss_lim[1]], 'sigma':[sigma, sigma_lim[0], sigma_lim[1]]}
    gaussian_dict = {'name': "Gaussian", 'label': "gaussian", 'parameters': gaussian}
    # Flat sky
    flatsky = {'I_sky': [Isky, Isky_lim[0], Isky_lim[1]]}
    flatsky_dict = {'name': "FlatSky", 'label': "flat_sky", 'parameters':flatsky}
    # flat bar
    flatbar = {'PA':[PA_ss, PA_lim[0],PA_lim[1]], 'ell':[ell_ss, ell_lim[0],ell_lim[1]],
               'deltaPA_max':[PA_ss, PA_lim[0],PA_lim[1]], 'I_0':[I_ss, Iss_lim[0],Iss_lim[1]],
               'h1':[h1, h_lim[0],h_lim[1]], 'h2':[h2, h_lim[0],h_lim[1]], 
               'r_break':[r_ss, rss_lim[0],rss_lim[1]], 'alpha':[alpha,alpha_lim[0],alpha_lim[1]]}
    flatbar_dict = {'name': "FlatBar", 'label': "flat_bar", 'parameters':flatbar}
    # Exponential
    exponential = {'PA': [PA_ss, PA_lim[0],PA_lim[1]], 'ell': [ell_ss, ell_lim[0],ell_lim[1]], 
                   'I_0': [I_ss, Iss_lim[0],Iss_lim[1]], 'h': [h1, h_lim[0],h_lim[1]]}
    exp_dict = {'name': "Exponential", 'label': "disk", 'parameters':exponential}
    return sersic_dict, psf_dict, gaussian_dict, flatsky_dict, flatbar_dict,exp_dict

def find_sky(imageAGNcrop, plothist=False):
    bgr = plt.hist(imageAGNcrop.flatten(),bins=np.arange(np.min(imageAGNcrop),20))
    plt.close()
    max_ind = np.where(bgr[0]==np.max(bgr[0]))[0][0]
    sky = (bgr[1][max_ind]+bgr[1][max_ind+1])/2
    if plothist:
        fig,ax = plt.subplots(1,2,figsize=(10,4))
        ax[0].hist(imageAGNcrop.flatten(),bins=np.arange(np.min(imageAGNcrop),np.max(imageAGNcrop)))
        ax[1].hist(imageAGNcrop.flatten(),bins=np.arange(np.min(imageAGNcrop),20))
        [ax[i].set_title(['Whole intensity range',f'sky level: {float(sky):.4f}'][i]) for i in range(2)]
        [ax[i].set_xlabel('intensity') for i in range(2)]
        ax[0].set_ylabel('number of agns') 
    return sky

def profile_1D(semiA,image,PA=180,ell=0.5):
    """make 1D elliptical profiles"""
    # create guess ellipse
    pos0 = image.shape[0]//2
    geometry = EllipseGeometry(x0=pos0, y0=pos0, sma=semiA, eps=ell,
                               pa=PA * np.pi / 180.0)
    # load image and geometry
    ellipse = Ellipse(image, geometry)
    # do isophote fit
    isolist = ellipse.fit_image()
    return isolist

def plot_isophotes(ax,isolist,num_aper=10):
    """plot aperatures on image"""
    for sma in np.linspace(isolist.sma[0],isolist.sma[-1],num_aper):
        iso = isolist.get_closest(sma)
        x, y, = iso.sampled_coordinates()
        ax.plot(x, y, color='white',linewidth="0.5")
        
def make_model_components(config,imshape):
    """make model component images from best fit config"""
    comp_names = config.functionLabelList()
    comp_ims=[]
    comp_pos = []
    for i in range(len(config.getModelAsDict()['function_sets'])):
        posX = config.getModelAsDict()['function_sets'][i]['X0']
        posY = config.getModelAsDict()['function_sets'][i]['Y0']
        functions = config.getModelAsDict()['function_sets'][i]['function_list']
        for j in range(len(functions)):
            if functions[j]['label'] =="bulge n=1" or functions[j]['label'] =="bulge n=4":
                functions[j]['parameters']['n'].append("fixed")
            funcset_dict = {'X0': posX, 'Y0': posY, 'function_list': [functions[j]]}
            model_dict = {'function_sets': [funcset_dict]}
            model = pyimfit.ModelDescription.dict_to_ModelDescription(model_dict)
            imfit_fitter = pyimfit.Imfit(model,epsf)
            comp_ims.append(imfit_fitter.getModelImage(shape=(imshape,imshape)))
            comp_pos.append([posX[0],posY[0]])
    return comp_ims, comp_pos, comp_names

def plot_model_components(comp_ims,comp_names,comp_pos,isolist_comps, colormap="ch:s=-.3,r=.6", plotIso=False,shrink_ratio=0.5):
    """plot 2D model components and check residual with model image"""
    clmap = sns.color_palette(colormap, as_cmap=True)
    ncom = len(comp_names)
    fig,ax = plt.subplots(nrows=1,ncols=ncom+1, figsize=(ncom*4,3))
    im = [ax[i].imshow(comp_ims[i],norm='symlog',cmap=clmap) for i in range(ncom)]
    [ax[i].text(0.05, 0.05, f"(x,y)=({comp_pos[i][0]:.1f},{comp_pos[i][1]:.1f})", transform=ax[i].transAxes, fontsize=8, color='k') for i in range(ncom-1)]
    [ax[i].set_title(comp_names[i]) for i in range(ncom)]
    im.append(ax[-1].imshow(np.sum(comp_ims[:-1],axis=0)-comp_ims[-1],norm='symlog',cmap=clmap))
    ax[-1].set_title("model-comps")
    [fig.colorbar(im[i], ax=ax[i], shrink=shrink_ratio).ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f')) for i in range(len(ax))]
    if plotIso:
        for i in range(len(isolist_comps)):
             plot_isophotes(ax[i],isolist_comps[i],num_aper=5)
    fig.tight_layout();

def plot_1isophote(ax,sma,isolist):
    """plot aperatures on image"""
    iso = isolist.get_closest(sma)
    x, y, = iso.sampled_coordinates()
    ax.plot(x, y, color='white',linewidth="0.3")

def pix_to_arcsec(imageFile,framelim):
    w = WCS(imageFile)
    # convert pixel to degree
    ra1,dec1 = w.pixel_to_world_values(0,0)
    ra2,dec2 = w.pixel_to_world_values(framelim,framelim)
    arcsec_per_pix = 16
    return arcsec_per_pix, [ra1,dec1,ra2,dec2]

def surface_brightness(intensity, area, magZPT):
    return -2.5*np.log10(intensity/area)+ magZPT

def radial_plot_params(imageFile, framelim, isolist_data,isolist_comps,hdu_exp,z=0.2):
    # convert pixel to arcsec and kpc
    arcsec_per_pix, skycoords = pix_to_arcsec(imageFile,framelim)
    sma_arcsec = isolist_data.sma*arcsec_per_pix
    sma_kpc = (cosmo.angular_diameter_distance(z)*sma_arcsec.to('rad').value).to('kpc')
    # calculate isophote areas and find surface brightness
    areas = (np.sqrt((1-isolist_data.eps**2)*sma_arcsec**2)*np.pi*sma_arcsec).value
    magZPT = hdu_exp.header['MAGZP']
    mu_data = [surface_brightness(i,areas,magZPT) for i in [isolist_data.intens,isolist_data.intens-isolist_data.int_err,isolist_data.intens+isolist_data.int_err]]
    mu_models = [[surface_brightness(i,areas,magZPT) for i in [isolist_comps[j].intens,isolist_comps[j].intens-isolist_comps[j].int_err,isolist_comps[j].intens+isolist_comps[j].int_err]] for j in range(len(isolist_comps))]
    return sma_arcsec, sma_kpc, mu_data, mu_models, skycoords

def plot_everything(on,image,m,modelname,comp_names,fsr,sma_arcsec,sma_kpc,mu_data,mu_models,skycoords,colormap):
    colors = sns.color_palette(colormap, len(comp_names)+2)[1:]
    linestyles = ['-', '--', '-.', ':']
    #colors = sns.color_palette("colorblind", len(comp_names)+2)
    cmapp = sns.color_palette(colormap, as_cmap=True).reversed()
    # Create grid and add subplots
    fig = plt.figure(figsize=(14, 4),layout='tight')
    gs = gridspec.GridSpec(2, 4, height_ratios=[3, 1], width_ratios=[1.25,1.25,1,1.5],hspace=0.05,wspace=0.05)
    ax1 = fig.add_subplot(gs[:, 0],xlabel='RA (deg)',ylabel='DEC (deg)') 
    ax2 = fig.add_subplot(gs[:, 1],xticks=[],yticks=[])
    ax3 = fig.add_subplot(gs[:, 2],xticks=[],yticks=[])
    ax4a = fig.add_subplot(gs[0, 3]) 
    ax4b = fig.add_subplot(gs[1, 3])   
    # formatting ticks
    xticks = np.linspace(skycoords[0],skycoords[2],4)
    yticks = np.linspace(skycoords[1],skycoords[3],4)
    ax1.set_xticks(np.linspace(0,image.shape[0],4))
    ax1.set_yticks(np.linspace(0,image.shape[0],4))
    ax1.set_xticklabels([f'{x:.3f}' for x in xticks])
    ax1.set_yticklabels([f'{y:.3f}' for y in yticks],rotation=90)
    ax1.tick_params(direction='in')
    # plot 2d and colorbars
    ax = [ax1,ax2,ax3,ax4a,ax4b]
    im = [ax[i].imshow([image, m][i], norm='symlog',cmap=cmapp) for i in range(2)]
    im2 = ax[2].imshow(image-m,cmap=cmapp)
    fig.colorbar(im2,ax=ax[2],orientation='horizontal',location='bottom',pad=0.05)
    fig.colorbar(im[1],ax=[ax[0],ax[1]],orientation='vertical',location='right',shrink=0.5)
    [ax[i].set_title([on,f"Model:\n{modelname}",f'Residual,$\chi^2_r$={fsr:.2f}'][i]) for i in range(3)]
    # radial plot data
    ax[3].plot(sma_arcsec[1:],mu_data[0][1:],label="data",c=colors[-1])
    ax[3].fill_between(sma_arcsec[1:].value,mu_data[1][1:],mu_data[2][1:],color=colors[-1],alpha=0.5)
    # radial plot components
    [ax[3].plot(sma_arcsec[1:],mu_models[i][0][1:],label=comp_names[i],linestyle=linestyles[i],c=colors[i]) for i in range(len(comp_names)-1)]
    [ax[3].fill_between(sma_arcsec[1:].value,mu_models[i][1][1:],mu_models[i][2][1:],color=colors[i],alpha=0.5) for i in range(len(comp_names)-1)]
    ax[4].plot(sma_kpc[1:],mu_data[0][1:]-mu_models[-1][0][1:],c=colors[-2],linestyle="dashdot")
    ax[4].fill_between(sma_kpc[1:].value,mu_data[1][1:]-mu_models[-1][1][1:],mu_data[2][1:]-mu_models[-1][2][1:],color=colors[-2],alpha=0.5)
    ax[4].axhline(y=0,linestyle='--',c=colors[1],lw=1)
    # format ticks
    ax[3].invert_yaxis()
    ax[3].set_xlabel("R[arcsec]")
    ax[3].set_ylabel("$\mu$ [mag arcsec$^{-2}$]")
    ax[3].xaxis.set_label_position('top') 
    ax[3].xaxis.set_ticks_position('top') 
    ax[3].legend(fontsize=10,loc='upper right',bbox_to_anchor=(1.6, 1))
    
    ax[4].set_xlabel("R[kpc]")
    ax[4].set_ylabel("$\Delta \mu$") 
    ax[4].set_ylim((-0.5,0.5))
    [ax.yaxis.set_label_position('right') for ax in [ax4a,ax4b]]
    [ax.yaxis.set_ticks_position('right') for ax in [ax4a,ax4b]];


def make_model_isophotes(isolist_data, comp_ims, comp_names, midf):
    isolist_comps=[]
    circ = [comp_names[i]=='psf' for i in range(len(comp_names))]
    for i in range(len(comp_ims)):
        isolist_ = []
        for iso in isolist_data[1:]:
            g = iso.sample.geometry
            ell = 0 if circ[i] else g.eps
            gn = EllipseGeometry(g.x0,g.y0, g.sma, ell, g.pa)
            sample = EllipseSample(comp_ims[i],g.sma,geometry=gn)
            sample.update()
            iso_ = Isophote(sample,0,True,0)
            isolist_.append(iso_)
        isolist = IsophoteList(isolist_)
        g = EllipseGeometry(midf,midf, 0.0, 0., 0.)
        sample = CentralEllipseSample(comp_ims[i], 0., geometry=g)
        fitter = CentralEllipseFitter(sample)
        center = fitter.fit()
        isolist.append(center)
        isolist.sort()
        isolist_comps.append(isolist)
    return isolist_comps

def make_data_isophotes(data,sma,midf):
    isolist_data = profile_1D(semiA=sma,image=data)
    # discard first isophote and make new
    isolist_data = isolist_data[1:]
    g = EllipseGeometry(midf,midf, 0.0, 0., 0.)
    sample = CentralEllipseSample(data, 0., geometry=g)
    fitter = CentralEllipseFitter(sample)
    center = fitter.fit()
    isolist_data.append(center)
    isolist_data.sort()
    return isolist_data

In [ ]:
from astropy.table import Table
hdul = fits.open("/home/insepien/research-data/alpaka/ALPAKA_v1_withDes.fits")
data = hdul[1].data
tb = Table(data).to_pandas()
tb.sort_values("RA", inplace=True)
# check mullaney for 2nd source
on = "J1140+0918"
ra = tb[tb['Desig'] == on].RA.values[0]
dec = tb[tb['Desig'] == on].DEC.values[0]
w = 0.8
ra_mask = (ra-w<tb.RA) & (tb.RA<ra+w)
dec_mask = (dec-w<tb.DEC) & (tb.DEC<dec+w)
tb[ra_mask & dec_mask][['Desig',"RA","DEC","AGN_TYPE"]]


In [ ]:
def make_peak_tbl(image,intens,agn_mask_size=35):
    """mask out agn and find peaks"""
    s = image.shape[0]
    midf = int(s//2)
    # mask out central agn
    peak_mask = np.zeros((s,s))
    peak_mask[midf-agn_mask_size:midf+agn_mask_size,midf-agn_mask_size:midf+agn_mask_size] = 1  
    peak_mask[20:30,20:40]=0
    # convert to boolean
    peak_mask = peak_mask==1
    # detect peaks
    peak_tbl = find_peaks(image,threshold=intens,mask=peak_mask)
    return peak_tbl


def make_mask(image,pos,aper_radius,pa=180):
    """make a mask provided position and aperture radius"""
    aper0 = EllipticalAperture(pos,aper_radius,aper_radius,pa)
    aper_mask0 = aper0.to_mask()
    mask0 = aper_mask0.to_image(image.shape)
    return mask0

def mask_image(image,rad,intens,agn_mask_size,PA=180):
    """create mask and masked images"""
    peak_tbl = make_peak_tbl(image,intens,agn_mask_size)
    mask=[]
    # make masks
    for i in range(len(peak_tbl)):
        mask.append(make_mask(image,pos = [peak_tbl[i]['x_peak'],peak_tbl[i]['y_peak']],aper_radius=rad,pa=PA))
    # sum all masks
    mask = np.sum(mask,axis=0)
    # make masked image
    masked_im = np.where(mask==0,image,0)
    return masked_im

on = "J1341-0049"
imageAGN = fits.getdata(glob.glob(os.path.expanduser(f"/home/insepien/research-data/agn-result/box/final_cut/{on}*.fits"))[0])
sky = find_sky(imageAGN,plothist=False)

# with open(os.path.expanduser("/home/insepien/research-data/agn-result/fit/final_fit_nb/"+on+".pkl"),"rb") as f:
#     d = pickle.load(f)
# imageAGN = d['imageSS']

imageAGN_masked = mask_image(image=imageAGN,rad=6,intens=5,agn_mask_size=25)-sky
plt.imshow(imageAGN_masked, norm='symlog')
# plt.plot(10,25,"wo")
# plt.plot(86,35,"wo")
print(sky)


In [ ]:
# # imageAGN_masked = imageAGN-sky
#fits.writeto("~/research-data/agn-result/fit/fit_masked_n.3to6/masked_image_SS/"+on+".fits",imageAGN_masked, overwrite=True)

In [ ]:
on = "J0813+0905"

imageFile = os.path.expanduser("~/research-data/agn-result/fit/fit_masked_n.3to6/masked_image_SS/"+on+".fits")
imageAGN = fits.getdata(os.path.expanduser(imageFile))
with open(os.path.expanduser("~/research-data/psf-results/psf_pkls/psf_"+on+".pkl"),"rb") as f:
    d = pickle.load(f)
epsf = d['psf'].data


ys,xs = find_highest_indices(imageAGN)
Imax = imageAGN.max()
itot=1500
framelim = imageAGN.shape[0]
midF=framelim//2

#sky = find_sky(imageAGN,plothist=True)
plt.imshow(imageAGN, norm='symlog')
plt.title(on)
# plt.plot(midF,midF,"ko",markersize=1)

In [ ]:
def _dofit_no_oversp(model, dataImage, psf, readnoise=0.22, expT=1, skylevel = 654.63, ncom=4, solver="NM",effgain=1):
    """do fit with not oversampled psf
       """
    fitter = pyimfit.Imfit(model,psf=psf)
    if float(ncom)==1.:
        fitter.loadData(dataImage,exp_time=expT, 
                    read_noise=readnoise, original_sky=skylevel,gain=effgain)
    else:
        fitter.loadData(dataImage,exp_time=expT, 
                    read_noise=readnoise, original_sky=skylevel,n_combined=ncom,gain=effgain)
    fitter.doFit(solver)
    fitConfig = fitter.getModelDescription()
    fitModelImage = fitter.getModelImage()
    fitResult = fitter.getFitResult()
    param_names = fitter.numberedParameterNames
    return fitConfig, fitModelImage, fitResult, param_names,fitter


Imax = imageAGN.max()
itot=1700
sersic_dict, psf_dict, gaussian_dict, flatsky_dict, flatbar_dict, exp_dict = makeModelDict(PA_ss=30, ell_ss=0.1, n_ss=1, I_ss=1, r_ss=20, Itot=itot,
                                                                     PA_lim=[0,360], ell_lim=[0.0,1.0], I_lim=[0.1,Imax],
                                                                     Iss_lim=[0.1,Imax], rss_lim=[0.1,framelim], Itot_lim=[0.1,1e4],
                                                                     sigma = 5, sigma_lim = [1,20], Isky = 2.5, Isky_lim =[0,10],
                                                                     h1=10,h2=10,h_lim=[0.1,framelim],alpha=0.1,alpha_lim=[0.1,framelim])
broken_exponentialParamsDict = {'PA': [0, 0,360], 'ell': [0.5, 0,1], 'I_0': [10.0, 0.0,Imax], 
                                'h1': [10, 0.1,framelim],'h2': [10, 0.1,framelim],'r_break': [10, 0.1,framelim],'alpha':[0.5,0,100]}
bexp_dict = {'name': "BrokenExponential", 'label': "bexp", 'parameters':broken_exponentialParamsDict}

GaussianRingAzParamsDict = {'PA': [40, 0,360], 'ell': [0.5, 0,1], 'A_maj':[100,0,framelim], 'A_min_rel':[0.5,0,1],
                          'R_ring':[10,0,100],'sigma_r':[5,0,100]}
gausR_dict = {'name': "GaussianRingAz", 'label': "GaussianRingAz", 'parameters': GaussianRingAzParamsDict}

exponentialParamDict = {'PA': [30, 0,360], 'ell': [0.5, 0,1], 'I_0': [1, 0.1,Imax], 'h': [1, 0.1,1e4]}
exp2_dict = {'name': "Exponential", 'label': "disk", 'parameters':exponentialParamDict}

flatbar = {'PA':[30,0,360], 'ell':[0.5,0,1],
            'deltaPA_max':[30,0,360], 'I_0':[1,0,Imax],
            'h1':[10,0.1,framelim], 'h2':[10,0.1,framelim,], 
            'r_break':[3,0,framelim], 'alpha':[0.1,0,framelim]}
flatbar0_dict = {'name': "FlatBar", 'label': "flat_bar", 'parameters':flatbar}


sersicParamsDict = {'PA': [30, 0, 360], 'ell_bulge': [0.5, 0, 1],
                    'n': [1, 0.3,6],'I_e': [10, 0.0,Imax], 'r_e': [5, 0.0, framelim]}
sersic2_dict = {'name': "Sersic", 'label': "bulge 2", 'parameters': sersicParamsDict}

sersic1_dict = {'name': "Sersic", 'label': "bulge 1", 'parameters': sersicParamsDict}

sersic3ParamsDict = {'PA': [30, 0, 360], 'ell_bulge': [0.7, 0, 1],
                    'n': [1,  0.3,6],'I_e': [10, 0.0,Imax], 'r_e': [3, 0.0, framelim]}
sersic3_dict = {'name': "Sersic", 'label': "bulge 3", 'parameters': sersic3ParamsDict}

ext_sersicParamsDict = {'PA': [30, 0, 360], 'ell_bulge': [0.5, 0, 1],
                    'n': [1,  0.3,6],'I_e': [10, 0.0,Imax], 'r_e': [5, 0.0, framelim]}
sersic_ext_dict = {'name': "Sersic", 'label': "ext bulge", 'parameters': ext_sersicParamsDict}


funcset_dict_sersic1= {'X0': [62,0,framelim], 'Y0': [63,0,framelim], 
                    'function_list': [sersic2_dict,psf_dict]}
funcset_dict_sersic2= {'X0': [62,0,framelim], 'Y0': [64,0,framelim], 
                    'function_list': [sersic_dict,psf_dict]}
funcset_dict_sersic3= {'X0': [52,0,framelim], 'Y0': [33,0,framelim], 
                    'function_list': [sersic_ext_dict]}



model_dict = {'function_sets': [funcset_dict_sersic1,funcset_dict_sersic3]}
model = pyimfit.ModelDescription.dict_to_ModelDescription(model_dict)

In [ ]:
fit_path = "/home/insepien/research-data/agn-result/fit/fit_masked_n.3to6/masked_fit/"+on+".pkl"
with open(os.path.expanduser(fit_path),"rb") as f:
    d_fit = pickle.load(f)

nb_path = "/home/insepien/research-data/agn-result/fit/final_fit_nb/"+on+".pkl"
with open(os.path.expanduser(nb_path),"rb") as f:
    d_nb = pickle.load(f)
d_fit['modelNames']['sersic+psf,sersic+psf'].getModelAsDict()

In [ ]:
mosfile = glob.glob(os.path.expanduser("~/raw-data-agn/mos-fits-agn/*"+on+"*.mos.fits"))[0]
expfile = glob.glob(os.path.expanduser("~/raw-data-agn/exp-fits-agn/*"+on+"*.exp.fits"))[0]
with fits.open(os.path.expanduser(mosfile)) as hdul:
    hdu0 = hdul[0]

sky_level = hdu0.header['BACKGND'] #fits:[e-/s] native pixels, pf: value should be in the same units as the data pixels

with fits.open(os.path.expanduser(expfile)) as hdul:
    hdu = hdul[0]
exptime= hdu.header['EXPOSURE'] # fits: actual exp time, pf: total integration time
gain = hdu.header['EGAIN'] #fits: no unit specified, pf: in electrons/ADU
noise=hdu.header['EFFRN'] #fits:[e-], pf: in electrons
numcom=hdu.header['NCOADD'] #Number of Averaged Frames   
# Try for only 1 model
image = imageAGN
config0, modelIm0, fitRes0, pname0,fitter0  = _dofit_no_oversp(model,dataImage=image, psf=epsf,solver="LM",
                                                 readnoise=noise, expT=exptime, skylevel = sky_level, ncom=numcom,effgain=gain)
m = modelIm0
res = fitRes0.params
pname = pname0
fs = fitRes0.fitStat
fsr = fitRes0.fitStatReduced
comp_ims, comp_pos, comp_names = make_model_components(config0,imshape=image.shape[0])
comp_ims.append(m)
comp_names.append("model")
comp_pos.append([midF,midF])
[print(f"{pname0[i]}: {fitRes0.params[i]:.2f} ") for i in range(len(fitRes0.params))];
fig,ax = plt.subplots(1,3,figsize=(10,3))
im0 = ax[0].imshow(image,norm='symlog')
im1=ax[1].imshow(np.sum(comp_ims,axis=0),norm='symlog')
im2=ax[2].imshow(image-m,norm='symlog')
[fig.colorbar([im0,im1,im2][i],ax=ax[i],shrink=0.5) for i in range(3)]
ax[0].set_title(on)
ax[1].set_title("")
ax[2].set_title(f'residual, $\chi$:{fs:.2f},$\chi_r$:{fitRes0.fitStatReduced:.2f}')
fig.tight_layout();
print(len(fitRes0.params))

In [ ]:
isolist_comps=[]
plot_model_components(comp_ims,comp_names,comp_pos,isolist_comps)

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(10,4))
im0=ax[0].imshow(comp_ims[0],norm='symlog')
im1=ax[1].imshow(image-m,norm='symlog')
[fig.colorbar([im0,im1][i],ax=ax[i],shrink=0.4) for i in range(2)]
fig.tight_layout();
ax[1].plot(comp_pos[0][0]-1,comp_pos[0][1]-1,"ro",markersize=2)
ax[1].plot(comp_pos[2][0]-1,comp_pos[2][1]-1,"ro",markersize=2)
# ax[1].plot(80,135,"ro",markersize=1)
#ax[1].plot(40,40,"ro",markersize=1)
#[print(f"{pname0[i]}: {fitRes0.params[i]:.2f} ") for i in range(len(fitRes0.params))];

In [ ]:
modelname = 'sersic+sersic+sersic'
isolist_data = make_data_isophotes(image,10,midF)
isolist_comps = make_model_isophotes(isolist_data,comp_ims,comp_names,midF)  
plot_model_components(comp_ims,comp_names,comp_pos,isolist_comps)
sma_arcsec, sma_kpc, mu_data,mu_models,skycoords = radial_plot_params(imageFile, framelim,isolist_data,isolist_comps,hdu_exp=hdu0,z=0.2)
plot_everything(on,image,m,modelname,comp_names,fsr,sma_arcsec,sma_kpc,mu_data,mu_models,skycoords,colormap="ch:s=-.3,r=.6")

In [ ]:
def save_data(modelname,image,model,configs,modelIms,fitResults,
              pnames,objectName, savepath):
    d = {}
    d[modelname] = model
    savedata = {}
    savedata['imageSS'] = image
    savedata['modelNames'] = d
    savedata['configs'] = configs
    savedata['modelImage'] = modelIms
    savedata['fitResults'] = fitResults
    savedata['paramNames'] = pnames
    pickle.dump(savedata,open(os.path.expanduser(savepath),"wb"))
    
outDir =os.path.expanduser("~/research-data/agn-result/fit/fit_masked_n.3to6/nb_fit/"+on+"_nb.pkl")
save_data(modelname,image=image, model=[model],configs=[config0],
          modelIms=[modelIm0],fitResults=[fitRes0],pnames=[pname0],
          objectName=on, savepath=outDir)



In [ ]:
from modelComponents import modelComps
on = "J0918+1100"

fit_path = "/home/insepien/research-data/agn-result/fit/fit_masked_n.3to6/masked_fit/"+on+".pkl"
comp_path = "/home/insepien/research-data/agn-result/fit/fit_masked_n.3to6/masked_fit_comp/"+on+"_comp.pkl"

with open(os.path.expanduser(fit_path),"rb") as f:
    d_fit = pickle.load(f)
with open(os.path.expanduser(comp_path),"rb") as f:
   d_comp = pickle.load(f)

modelname = 'sersic+psf,sersic'
# print parameters
ind = np.where(np.array(list(d_fit['modelNames'].keys()))==modelname)[0][0]
print(f"chi_r:{d_fit['fitResults'][ind].fitStatReduced:.5f}")
[print(f"{d_fit['paramNames'][ind][i]}:{d_fit['fitResults'][ind].params[i]:.2f}") for i in range(len(d_fit['paramNames'][ind]))];

plt.imshow(d_fit['imageSS'],norm='symlog')


In [ ]:
df2 = pd.read_pickle("/home/insepien/research-data/pop-result/separation.pkl")
sep_kpc = df2['sep_kpc']

def find_agn_in_bin(bin_num):
    bins = np.logspace(-1,1.5,10)
    bin0_inds = np.where((bins[bin_num] <= sep_kpc) & (sep_kpc <= bins[bin_num+1]))
    bin0_agns = df2.loc[[i if i<14 else i+1 for i in bin0_inds[0]]]
    return bin0_agns

find_agn_in_bin(2)

In [ ]:
df_all = pd.read_pickle("/home/insepien/research-data/pop-result/all_results_updated.pkl")
with open(os.path.expanduser("flag.txt"),"r") as f:
    d = f.read().splitlines()
df_all['flag'] = d[1:]
serind = []
df_alll = df_all[df_all['flag']=="y"]
[[serind.append(df_alll['sersic index'].values[i][j]) for j in range(len(df_alll['sersic index'].values[i]))] for i in range(len(df_alll))]
plt.hist(serind);

mask = [np.sum(np.array(df_all['sersic index'].values[i])>2)!= 1 for i in range(len(df_all))]
mask2 = df_all['flag']=="y"
df_all[mask & mask2][['Obj Name', "sersic index", "flag"]]

In [ ]:
## J1010 CHECKS
from astropy.coordinates import SkyCoord
from astropy.table import Table
data = fits.getdata('/home/insepien/research-data/alpaka/ALPAKA_v1.fits')
def find1010(final_table):
    """find j1010 in alpaka"""
    for i in range(0, len(final_table)):
        pos = SkyCoord(ra=final_table['RA'][i]*u.deg, dec=final_table['DEC'][i]*u.deg)
        posstring = pos.to_string('hmsdms').split(' ')
        des_ra = posstring[0][0:2]+posstring[0][3:5]
        des_dec = posstring[1][0:3]+posstring[1][4:6]
        des = 'J'+des_ra+des_dec
        if des == "J1010+1413":
            return i
ind = find1010(data) 
# get j1010 row
j1010 = Table(data).to_pandas().loc[ind].to_dict()
lums = [i for i in list(j1010.keys()) if 'LUM' in i]

# OIII
ja_oiii = 1.2e44 # different from alpaka value??
i1 = ja_oiii*800/(1+1.3)
i2 = i1*1.3
print(f"alpaka OIII lum extinction corrected: {j1010['OIII_5007_LUM_DERRED']:.1e}")
print(f"paper OIII lum: {ja_oiii:.1e}")
print(f"by paper lum, core 1 bolometric: {i1:.1e}, core 2 bolometric: {i2:.1e}")

# IR -- use Planck13 here
f160 = 3.23e-12*u.erg/u.s/(u.cm)**2 # from paper
ja_ir = 1.1e11
da = cosmo.luminosity_distance(j1010['Z'])
print(f"paper IR lum: {ja_ir:.1e}")
print(f"IR Lum: {(f160*4*np.pi*da**2).to(u.L_sun):.1e}")

In [ ]:
from astropy.cosmology import Planck18 as cosmo
mylum = (obs_flux*obs_hz*4*np.pi*cosmo.luminosity_distance(0.2)**2).to(u.erg/u.s).value
print(f"paper IR lum: {ja_ir:.1e}")
print(f"IR Lum: {mylum:.1e}")

In [ ]:
# checking J1010 WISE numbers
from astropy.cosmology import Planck13 as cosmo
import astropy.constants as const
wmags_ = np.array([13.056, 11.208, 6.784, 3.816])

zeromagflux = np.array([309.540, 171.787, 31.674, 8.363])*u.Jy
fluxdens = zeromagflux*10**(-wmags_/2.5)
## get flux at obs wavelength
obs_flux = fluxdens[-1]
## change to luminosity
obs_hz = (const.c/(22*u.micron)).to(u.Hz)
ja_ir = 6e46
mylum = (obs_flux*obs_hz*4*np.pi*cosmo.luminosity_distance(0.2)**2).to(u.erg/u.s).value
print(f"paper IR lum: {ja_ir:.1e}")
print(f"IR Lum: {mylum:.1e}")

In [ ]:
from scipy.optimize import fsolve
import astropy.constants as const

def wise_lum_from_mag(wmags_, wmags_err_, rf_wavelength_, redshift_):
    """turn wise mags into wise flux at restframe 15 microns"""
    ## get observed wavelength corresponding to global rest-frame wavelength
    obs_wavelength = rf_wavelength_*(1+redshift_)
  
    ## change mags to fluxes -- http://wise2.ipac.caltech.edu/docs/release/allsky/expsup/sec4_4h.html#example
    zeromagflux = np.array([309.540, 171.787, 31.674, 8.363])*u.Jy
    fluxdens = zeromagflux*10**(-wmags_/2.5)
    fluxdens_err = zeromagflux*10**(-wmags_err_/2.5)
  
    ## interpolate - use straight line
    wise_wavelengths = np.array([3.4, 4.6, 12., 22.])
    wiseflux = np.polyfit(wise_wavelengths, fluxdens.value,1, w=1./fluxdens_err)
  
  
    ## get flux at obs wavelength
    obs_flux = (wiseflux[0]*obs_wavelength+wiseflux[1])*u.Jy
  
    ## change to luminosity
    obs_hz = (const.c/(obs_wavelength*u.micron)).to(u.Hz)
    lum = (obs_flux*obs_hz*4*np.pi*
           cosmo.luminosity_distance(redshift_)**2).to(u.erg/u.s)
    return lum


wise = pd.read_pickle("/home/insepien/research-data/alpaka/wise_39fits.pkl")
wise.fillna(0,inplace=True)
#wise = ascii.read('/home/insepien/research-data/alpaka/table_irsa_catalog_search_results_41.tbl', format='ipac')
alpaka = pd.read_pickle("/home/insepien/research-data/alpaka/alpaka_39fits.pkl")
alpaka.reset_index(inplace=True)

## get wise mags and errors
w1mag = wise['w1mpro']
w2mag = wise['w2mpro']
w3mag = wise['w3mpro']
w4mag = wise['w4mpro']
wmags = np.array([w1mag, w2mag, w3mag, w4mag])
wmags_err = np.array([wise['w1sigmpro'], wise['w2sigmpro'], wise['w3sigmpro'], wise['w4sigmpro']])

# calculate 15 microns lum
wise_lums = np.zeros((len(wise)))
for i in range(0, len(wise)):
    on = wise.loc[i,"designation"]
    z = np.mean(alpaka[alpaka['Desig']==on]['Z'])
    wise_lums[i] = wise_lum_from_mag(wmags[:,i], wmags_err[:,i], 15., alpaka.loc[i,"Z"]).value


In [ ]:
# bolometric correction from Hopkins
with open("bc.txt","r") as f:
    d = f.read().splitlines()
df = pd.DataFrame([d[1:][i].split(' ') for i in range(len(d[1:]))],columns=d[0].split(' '))
Lbol = np.array(list(df['Lbols'].values), dtype=float)
LIR = np.array(list(df['LIRs'].values), dtype=float)

# interpolate wise IR lum
from scipy.interpolate import CubicSpline
spl = CubicSpline(LIR, Lbol)
irbol = 10**(spl(np.log10(wise_lums)))

In [ ]:
# compare OIII and IR bolo lum
df_all = pd.read_pickle("/home/insepien/research-data/pop-result/all_results_updated.pkl")
oiiibol = np.array(list(df_all['OIII_BOL'].values))
plt.hist(np.log10(oiiibol),label="oiii")
plt.hist(np.log10(irbol),label='ir')
plt.xlabel("Log(L_bol)")
plt.legend();

In [ ]:
plt.scatter(np.log10(oiiibol), np.log10(irbol))
plt.plot(np.log10(oiiibol), np.log10(oiiibol), label="oiii=ir")
plt.plot(np.log10(oiiibol), np.log10(oiiibol/100), label="oiii/100=ir")
plt.legend()
plt.xlabel("Log( OIII L_bol )")
plt.ylabel("Log( IR L_bol )")


In [ ]:
plt.hist(np.array(list(df_all['OIII_BOL'].values))/irbol,bins=[0.1,1,10,100,1000])
plt.xscale("log")
plt.xlabel("Lbol_OIII / Lbol_IR")